In [ ]:
import json
import string
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')
table = str.maketrans('','',string.punctuation)
with open("Sarcasm_Headlines_Dataset.json", 'r') as f:
  datastore = [json.loads(line) for line in f]
sentences = []
labels = []
urls = []
for item in datastore:
  sentence = item['headline'].lower()
  sentence = sentence.replace(",", " , ")
  sentence = sentence.replace(".", " . ")
  sentence = sentence.replace("-", " - ")
  sentence = sentence.replace("/", " / ")
  soup = BeautifulSoup(sentence)
  sentence = soup.get_text()
  words = sentence.split()
  filtered_sentence = ""
  for word in words:
   word = word.translate(table)
   if word not in stopwords:
     filtered_sentence = filtered_sentence + word + " "
  sentences.append(filtered_sentence)
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])
  training_size = 25000
tr_sentences = sentences[0:training_size]
tr_labels = labels[0:training_size]
t_sentences = sentences[training_size:]
t_labels = labels[training_size:]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Embedding

In [ ]:
model = Sequential([
                    Embedding(20000,7),
                    Bidirectional(LSTM(7)),
                    Dense(24,activation = 'relu'),
                    Dense(1,activation = 'sigmoid')
])

In [ ]:
adam_new = tf.keras.optimizers.Adam(learning_rate = 0.00001,beta_1=0.9,beta_2 =0.999,amsgrad=False)

In [ ]:
model.compile(loss = 'binary_crossentropy',optimizer = adam_new,metrics = ['accuracy'])

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000,oov_token = '<OOV')

In [ ]:
tokenizer.fit_on_texts(tr_sentences)

In [ ]:
tr_seq= tokenizer.texts_to_sequences(tr_sentences)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tr_padded = pad_sequences(tr_seq,maxlen=10,truncating='post',padding='post')

In [ ]:
import numpy as np
tr_padded = np.array(tr_padded)
tr_labels = np.array(tr_labels)

In [ ]:
print(tr_padded)

[[  218 13789   596 ...  1903  2325  7873]
 [ 7874  3374  2670 ...   984     0     0]
 [   72   761   697 ...   135  9990     0]
 ...
 [   65  6975  2859 ...     0     0     0]
 [ 3749  3720  3558 ...   198  1655     0]
 [  404  1300     1 ...   555     0     0]]


In [ ]:
model.fit(tr_padded,tr_labels,epochs =50)

Epoch 1/50
782/782 [==============================] - 8s 10ms/step - loss: 0.6851 - accuracy: 0.5609
Epoch 2/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6832 - accuracy: 0.5609
Epoch 3/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6810 - accuracy: 0.5609
Epoch 4/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6787 - accuracy: 0.5610
Epoch 5/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6760 - accuracy: 0.5612
Epoch 6/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6731 - accuracy: 0.5620
Epoch 7/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6697 - accuracy: 0.5647
Epoch 8/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6657 - accuracy: 0.5721
Epoch 9/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6612 - accuracy: 0.5822
Epoch 10/50
782/782 [==============================] - 7s 9ms/step - loss: 0.6560 - accuracy: 0.596

In [ ]:
t_deq = tokenizer.texts_to_sequences(t_sentences)

In [ ]:
t_padded = pad_sequences(t_deq,maxlen=10,truncating='post',padding='post')

In [ ]:
t_padded = np.array(t_padded)
t_labels = np.array(t_labels)

In [ ]:
model.evaluate(t_padded,t_labels)

54/54 [==============================] - 1s 2ms/step - loss: 0.4616 - accuracy: 0.7999


[0.46159443259239197, 0.7998829483985901]

In [ ]:
import tensorflow as tf
model2 = tf.keras.Sequential([
 tf.keras.layers.Embedding(20000,7 ),
 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(7,
return_sequences=True)),
 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(7)),
 tf.keras.layers.Dense(24, activation='relu'),
 tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model2.compile(loss='binary_crossentropy',optimizer = adam_new,metrics = ['accuracy'])

In [ ]:
model2.fit(tr_padded,tr_labels,epochs = 25)

Epoch 1/25
782/782 [==============================] - 11s 14ms/step - loss: 0.6793 - accuracy: 0.5609
Epoch 2/25
782/782 [==============================] - 11s 14ms/step - loss: 0.6760 - accuracy: 0.5609
Epoch 3/25
782/782 [==============================] - 12s 16ms/step - loss: 0.6717 - accuracy: 0.5610
Epoch 4/25
782/782 [==============================] - 11s 14ms/step - loss: 0.6657 - accuracy: 0.5637
Epoch 5/25
782/782 [==============================] - 11s 14ms/step - loss: 0.6576 - accuracy: 0.5775
Epoch 6/25
782/782 [==============================] - 11s 14ms/step - loss: 0.6463 - accuracy: 0.6098
Epoch 7/25
782/782 [==============================] - 11s 14ms/step - loss: 0.6308 - accuracy: 0.6544
Epoch 8/25
782/782 [==============================] - 11s 14ms/step - loss: 0.6101 - accuracy: 0.7030
Epoch 9/25
782/782 [==============================] - 11s 14ms/step - loss: 0.5832 - accuracy: 0.7404
Epoch 10/25
782/782 [==============================] - 11s 14ms/step - loss: 0.551

In [ ]:
model2.evaluate(t_padded,t_labels)

54/54 [==============================] - 2s 4ms/step - loss: 0.4916 - accuracy: 0.7765


[0.4915625751018524, 0.7764774560928345]